In [ ]:
import sys,os, pandas as pd,datetime,glob
from pathlib import Path
project_root = os.path.abspath(Path("../."))
sys.path.append(str(project_root))
results_folder = str(project_root) + '/results/performance2/baselines20251214/'

In [2]:

list_results_files = glob.glob(f"{results_folder}*.txt")
list_items = []
for file in list_results_files:
    with open(file,'r') as f:
        data = f.read()
    data = data.replace("][","],[").replace('inf','0').replace('nan','0').replace('fi0cial_metrics','financial_metrics')
    data = eval(data)
    for items in data:
        for item in items:
            list_items.append(item)
            # print(item)
periods =['training','validating','testing']
seeds = [10,200,3000,40000,500000]
list_TargetIsClose = [True,False]
print(list_items)
df=pd.DataFrame(list_items)
df = df[(df['GLOBAL_SEED'].isin(seeds) & df['period'].isin(periods))]
df = df.drop('at',axis=1)
df = df.drop_duplicates()
df[['baseline_modelName','GLOBAL_SEED']].value_counts().sort_index()

[{'GLOBAL_SEED': 10, 'baseline_modelName': 'baseline_BuyAndHold', 'target_isClose': True, 'period': 'training', 'rmse': 780.616931906994, 'mae': 716.2540588940235, 'r2': -5.324936074250366, 'strategy_name': 'baseline_BuyAndHold', 'strategy_type': 'moc', 'n_days': 3262, 'trading_days': 3261, 'skipped_trades': 0, 'gap_skips': 0, 'edge_skips': 0, 'trade_rate': 0.9996934396076027, 'win_days': 1679, 'win_rate': 0.5148727384237963, 'compound_gross_return': 1.331388067276047, 'total_return': 1.0769871552611778, 'annualized_return': 0.06757749178506645, 'cagr': 0.06744424939914984, 'sharpe_ratio': 0.441307361189705, 'sortino_ratio': 0.4313074310097893, 'omega_ratio': 1.082454280930039, 'max_drawdown': 0.443638441258952, 'calmar_ratio': 0.1523256000839238, 'profit_factor': 1.082454280930039, 'volatility': 0.18853237393656844, 'risk_free_rate': 0.0, 'execution_threshold': 0.0, 'max_gap': 0.02, 'at': '2025-12-13 20:44:16.190724'}, {'GLOBAL_SEED': 10, 'baseline_modelName': 'baseline_BuyAndHold', '

baseline_modelName   GLOBAL_SEED
baseline_ARIMA       10             6
                     200            6
                     3000           6
                     40000          6
                     500000         6
baseline_BuyAndHold  10             6
                     200            6
                     3000           6
                     40000          6
                     500000         6
baseline_EAF         10             6
                     200            6
                     3000           6
                     40000          6
                     500000         6
baseline_LGBM        10             6
                     200            6
                     3000           6
                     40000          6
                     500000         6
baseline_LSTM        10             6
                     200            6
                     3000           6
                     40000          6
                     500000         6
baseline_XGBM    

In [3]:
numberic_cols = df.select_dtypes('number').columns.to_list()
group_cols=['baseline_modelName','target_isClose','period']
drop_cols =['GLOBAL_SEED','n_days','skipped_trades','gap_skips','edge_skips','compound_gross_return','win_days','risk_free_rate','execution_threshold','max_gap']
# group_cols+numberic_cols
df_raw = df[group_cols+numberic_cols]
df_raw.to_csv(results_folder +'/raw_outputs.csv',index=False)
df_group = df[group_cols+numberic_cols].groupby(group_cols).mean().drop(columns=drop_cols).reset_index()
df_group['target_isClose'] = df_group['target_isClose'].apply(lambda x: 'Close' if x else 'LogReturn')
df_group=df_group.rename({'target_isClose':'target'},axis=1)
df_group = df_group.sort_values(['period','target','annualized_return'],axis=0, ascending=False).reset_index(drop=True)
df_group.to_csv(results_folder +'/summary_outputs.csv',index=False)
df_group

,baseline_modelName,target,period,rmse,mae,r2,trading_days,trade_rate,win_rate,total_return,annualized_return,cagr,sharpe_ratio,sortino_ratio,omega_ratio,max_drawdown,calmar_ratio,profit_factor,volatility
0,baseline_EAF,LogReturn,validating,1.387004,0.972963,-0.781048,416.4,0.550066,0.554003,0.407848,0.139398,0.139834,1.276808,0.996435,1.373657,0.108481,1.521171,1.373657,0.106544
1,baseline_LSTM,LogReturn,validating,1.159925,0.838260,-0.228966,304.0,0.406961,0.587435,0.381005,0.131473,0.131824,1.239713,0.772962,1.446047,0.140078,1.159505,1.446047,0.104417
2,baseline_XGBM,LogReturn,validating,1.038485,0.708738,0.003426,416.4,0.550066,0.557143,0.381366,0.128560,0.128960,1.142159,0.846662,1.335649,0.112951,1.376706,1.335649,0.112657
3,baseline_BuyAndHold,LogReturn,validating,1.041235,0.709546,-0.001859,756.0,0.998679,0.552910,0.387772,0.122417,0.122796,0.783419,0.733260,1.156381,0.182208,0.671854,1.156381,0.164773
4,baseline_LGBM,LogReturn,validating,1.177115,0.859833,-0.280598,224.2,0.296169,0.572683,0.277651,0.092401,0.092684,1.004981,0.544956,1.449170,0.125231,0.771726,1.449170,0.091895
5,baseline_ARIMA,LogReturn,validating,1.041275,0.709626,-0.001934,136.0,0.179657,0.580882,0.174465,0.058591,0.058767,1.218299,0.526102,1.649138,0.047594,1.231063,1.649138,0.047672
6,baseline_BuyAndHold,Close,validating,447.870370,395.450200,-3.537468,756.0,0.998679,0.552910,0.387772,0.122417,0.122796,0.783419,0.733260,1.156381,0.182208,0.671854,1.156381,0.164773
7,baseline_EAF,Close,validating,80.247993,52.430606,0.854183,327.2,0.432232,0.575118,0.362331,0.122345,0.122725,1.193312,0.796349,1.404925,0.078997,1.548743,1.404925,0.103531
8,baseline_LGBM,Close,validating,84.908036,62.977188,0.836896,216.6,0.286129,0.526454,0.097939,0.029565,0.029653,0.389110,0.191370,1.146499,0.131603,0.224631,1.146499,0.083605
9,baseline_LSTM,Close,validating,140.547221,120.291759,0.536701,25.0,0.033467,0.627558,0.082841,0.027725,0.027796,0.858783,0.313463,3.858305,0.018794,2.082562,3.858305,0.034274
